In [1]:
# !pip install tensorflow

In [1]:
# import tensorflow as tf
# from tensorflow.keras import layers, models
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.utils import to_categorical

## Loading imsg

In [7]:
# !pip install opencv-python

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
datagen = ImageDataGenerator(rescale=1./255) # Create an instance of ImageDataGenerator with rescaling
data_folder_path = 'RawData/'

image_height = 128
image_width = 128
batch_size = 32

# Create a generator for training data
train_generator = datagen.flow_from_directory(
    data_folder_path,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical'  # Set class_mode to 'categorical'
)

Found 19867 images belonging to 4 classes.


In [20]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(train_generator[0][0], train_generator[0][1], test_size=0.1, random_state=42)

# Further split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)

In [21]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(24, 128, 128, 3) (24, 4)
(4, 128, 128, 3) (4, 4)
(4, 128, 128, 3) (4, 4)


## CNN model

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [23]:
# Define the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(image_height, image_width, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4, activation='softmax'))  # 4 classes


In [24]:
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 128)     

In [26]:
# Train the model using the image data generator
epochs = 1
history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))

1/1 [==============================] - 1s 868ms/step - loss: 1.3326 - accuracy: 0.4583 - val_loss: 1.0704 - val_accuracy: 0.5000


In [27]:
# Evaluate the model on the testing set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

1/1 [==============================] - 0s 48ms/step - loss: 0.9689 - accuracy: 0.5000
Test Accuracy: 50.00%


In [28]:
# Save the trained model
model.save('cnn_v1.h5')